In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import os
import numpy as np
import pandas as pd
import squidpy as sq

import seaborn as sns
import anndata as ad
import matplotlib.colors as mcolors
# from preprocess_helpers import get_scVI_latent_representation, compute_local_mean, get_LR_pattern_representation, clean_genes_for_LR_GSE232025



In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import numpy as np
import pandas as pd
import squidpy as sq
import os
from sklearn.preprocessing import LabelEncoder
import scprep
from scvi.model import SCVI
import torch
import torchsde
from torchdyn.core import NeuralODE
from tqdm import tqdm
from torchcfm.conditional_flow_matching import *
from torchcfm.models import MLP, GradModel
from torchcfm.utils import plot_trajectories, torch_wrapper
import pandas as pd
import seaborn as sns
from umap import UMAP
import torch.nn as nn
from torchdyn.core import NeuralODE
import seaborn as sns
from scvi.model import SCVI
import scvi
from scipy.stats import wasserstein_distance
from scipy.stats import energy_distance
from sklearn.metrics import r2_score
from sklearn.neighbors import NearestNeighbors
from scipy import sparse
import liana as li
import re


def compute_local_mean(scRNA, representation='X_pca', spatial_key='spatial', radius=50):
    """
    Compute mean vector of the chosen representation for spatial neighbors.
    
    Parameters:
        scRNA: AnnData object
        representation: str, key in .obsm, e.g. 'X_pca', 'X_umap', 'X_scVI'
        spatial_key: str, key in .obsm with spatial coordinates
        radius: float, neighborhood radius in same units as coordinates
    """
    coords = scRNA.obsm[spatial_key]
    X = scRNA.obsm[representation]

    nbrs = NearestNeighbors(radius=radius).fit(coords)
    neighbors_idx = nbrs.radius_neighbors(coords, return_distance=False)

    local_means = np.zeros_like(X)

    for i, idx in enumerate(neighbors_idx):
        if len(idx) > 0:
            local_means[i] = X[idx].mean(axis=0)
        else:
            local_means[i] = X[i]  

    # Store the result in .obsm
    scRNA.obsm[f"local_mean_{representation}"] = local_means

    return scRNA


def get_scVI_latent_representation(scRNA, cell_type_key, spatial_key):
        print ("Using scVI for input data")
        #read the data (again cause scVI requires unormalized data) and set up scVI
        le = LabelEncoder()
        scRNA.obs[cell_type_key] = le.fit_transform(scRNA.obs[cell_type_key])
        scvi.model.SCVI.setup_anndata(scRNA)
        model = SCVI(scRNA)
        model.train()
        latent = model.get_latent_representation()
        scRNA.obsm["X_scVI"] = latent
        scRNA = compute_local_mean(scRNA, representation='X_scVI', radius=50)
        return scRNA



def get_LR_pattern_representation(scRNA, nz_prop=0.1, resource_name='consensus'):

    li.ut.spatial_neighbors(scRNA, bandwidth=200, cutoff=0.1, kernel='gaussian', set_diag=True)
    lrdata_by_day= []
    for stage in scRNA.obs['day'].unique():
        adata_stage = scRNA[scRNA.obs['day'] == stage].copy()
        lrdata = li.mt.bivariate(adata_stage,
                    resource_name=resource_name, # NOTE: uses HUMAN gene symbols!
                    local_name='cosine', # Name of the function
                    global_name="morans", # Name global function
                    n_perms=100, # Number of permutations to calculate a p-value
                    mask_negatives=False, # Whether to mask LowLow/NegativeNegative interactions
                    add_categories=True, # Whether to add local categories to the results
                    nz_prop=nz_prop, # Minimum expr. proportion for ligands/receptors and their subunits
                    use_raw=False,
                    verbose=True
                    )
        # Save or analyze results for this stage
        # adata_stage.uns['liana_res'].to_csv(f'liana_results_stage_{stage}.csv')
        lrdata_by_day.append(lrdata)


    

    # Step 1: Collect all unique LR pairs across all time points
    all_lr_pairs = set()
    for lrdata in lrdata_by_day:
        all_lr_pairs.update(lrdata.var_names)

    # Convert to sorted list for consistent ordering
    all_lr_pairs = sorted(list(all_lr_pairs))
    print(f"Total unique LR pairs across all time points: {len(all_lr_pairs)}")

    # Step 2: Create LR_pattern feature vectors for each time point
    for i, stage in enumerate(scRNA.obs['day'].unique()):
        # Get the subset of cells for this stage
        stage_mask = scRNA.obs['day'] == stage
        n_cells_stage = stage_mask.sum()
        
        # Initialize LR_pattern matrix with zeros
        lr_pattern_matrix = np.zeros((n_cells_stage, len(all_lr_pairs)))
        
        # Get the lrdata for this stage
        lrdata = lrdata_by_day[i]
        
        # Convert sparse matrix to dense if needed
        if sparse.issparse(lrdata.X):
            lrdata_dense = lrdata.X.toarray()
        else:
            lrdata_dense = lrdata.X
        
        # Fill in the values for LR pairs that exist in this stage
        for j, lr_pair in enumerate(all_lr_pairs):
            if lr_pair in lrdata.var_names:
                # Get the index of this LR pair in the current lrdata
                lr_idx = lrdata.var_names.get_loc(lr_pair)
                # Copy the values for all cells
                lr_pattern_matrix[:, j] = lrdata_dense[:, lr_idx]
            # If LR pair doesn't exist in this stage, it remains 0 (already initialized)
        
        # Add LR_pattern to obsm for cells of this stage
        if 'LR_pattern' not in scRNA.obsm:
            # Initialize the full LR_pattern matrix for all cells
            scRNA.obsm['LR_pattern'] = np.zeros((scRNA.n_obs, len(all_lr_pairs)))
        
        # Fill in the values for this stage
        scRNA.obsm['LR_pattern'][stage_mask] = lr_pattern_matrix
        
        print(f"Stage {stage}: {n_cells_stage} cells, {len(lrdata.var_names)} LR pairs available")

    # Step 3: Add metadata about the LR pairs
    scRNA.uns['LR_pattern_pairs'] = all_lr_pairs
    print(f"LR_pattern matrix shape: {scRNA.obsm['LR_pattern'].shape}")
    print(f"LR pair names stored in scRNA.uns['LR_pattern_pairs']")

    return scRNA


def clean_genes_for_LR_GSE232025(scRNA):
    scRNA.var["gene_symbol_original"] = scRNA.var_names
    def clean_name(name):
        # keep leftmost alias before '|'
        primary = name.split("|")[0]
        # remove [hs], [nr], etc.
        primary = re.sub(r"\[.*?\]", "", primary)
        return primary.strip()
    scRNA.var_names = [clean_name(x) for x in scRNA.var_names]
    scRNA.var_names_make_unique()
    return scRNA

In [43]:
# import os, subprocess
# os.makedirs("MOSTA", exist_ok=True)
# # list of stages
# stages = ["E9.5", "E10.5", "E11.5", "E12.5", "E13.5", "E14.5", "E15.5", "E16.5"]
# # base URL pattern
# base_url = "https://ftp.cngb.org/pub/SciRAID/stomics/STDS0000058/stomics/{stage}_E1S1.MOSTA.h5ad"
# for stage in stages:
#     url = base_url.format(stage=stage)
#     out = f"MOSTA/{stage}.h5ad" 
#     print(f"Downloading {url} → {out}")
#     subprocess.run([
#         "wget", "--progress=bar:force", "-O", out, url
#     ])

In [44]:
def load_scRNA_GSE062025(data_path):
    """
    This function processes the GSE062025 dataset and returns the concatenated AnnData object.
    """
    sc1= sc.read_h5ad(os.path.join(data_path, "E9.5.h5ad"))
    sc1.obs["Batch"] = "0"
    sc2= sc.read_h5ad(os.path.join(data_path, "E10.5.h5ad"))
    sc2.obs["Batch"] = "1"
    sc3= sc.read_h5ad(os.path.join(data_path, "E11.5.h5ad"))
    sc3.obs["Batch"] = "2"
    sc4= sc.read_h5ad(os.path.join(data_path, "E12.5.h5ad"))
    sc4.obs["Batch"] = "3"
    sc5= sc.read_h5ad(os.path.join(data_path, "E13.5.h5ad"))
    sc5.obs["Batch"] = "4"
    # sc6= sc.read_h5ad(os.path.join(data_path, "E14.5.h5ad"))
    # sc6.obs["Batch"] = "5"
    sc7= sc.read_h5ad(os.path.join(data_path, "E15.5.h5ad"))
    sc7.obs["Batch"] = "6"
    sc8= sc.read_h5ad(os.path.join(data_path, "E16.5.h5ad"))
    sc8.obs["Batch"] = "7"

    # adatas = [sc1, sc2, sc3, sc4, sc5, sc6, sc7, sc8]
    adatas = [sc1, sc2, sc3, sc4, sc5, sc7, sc8]
    ## cell color mapping
    all_cell_types = set()
    for adata in adatas:
        all_cell_types.update(adata.obs["annotation"].unique())

    # Create a consistent color map for all cell types
    palette = sns.color_palette("tab20", len(all_cell_types))
    color_dict = dict(zip(sorted(all_cell_types), palette))

    for sci in adatas:
        sci.obs = sci.obs.rename(columns={"annotation": "celltype", "Batch": "day"})
        sci.obs["celltype"] = sci.obs["celltype"].astype(str)
        sci.obs["color"] = sci.obs["celltype"].map(color_dict)
        sci.obs["color"] = sci.obs["color"].apply(lambda x: mcolors.to_hex(x) if isinstance(x, (tuple, list, np.ndarray)) else x)
    scRNA = ad.concat(adatas, label='day', keys=[a.obs['day'][0] for a in adatas])

    return scRNA

In [ ]:
data_path= "/Users/rssantanu/Desktop/codebase/constrained_FM/scratch/MOSTA"

scRNA = load_scRNA_GSE062025(data_path)

/var/folders/mc/s2w117k11cj56552qbkf28zw0000gn/T/ipykernel_12370/1852424913.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`


In [ ]:
num_genes = 10000
n_comps = 50

In [ ]:
sc.pp.highly_variable_genes(scRNA, batch_key='day', n_top_genes=num_genes, flavor='seurat_v3')
scRNA = scRNA[:, scRNA.var.highly_variable]
sc.tl.pca(scRNA, n_comps=n_comps)
sc.pp.neighbors(scRNA)
sc.tl.umap(scRNA)
# scRNA= compute_local_mean(scRNA, representation='X_pca')
# scRNA = get_scVI_latent_representation(scRNA, cell_type_key='celltype', spatial_key='spatial')

# scRNA = clean_genes_for_LR_GSE232025(scRNA)
# scRNA = get_LR_pattern_representation(scRNA, nz_prop=nz_prop)

/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:75: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:172: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns["hvg"] = {"flavor": flavor}
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:314: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm[key_obsm] = X

In [ ]:
scRNA

AnnData object with n_obs × n_vars = 418296 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'celltype', 'Regulon - Arx', 'Regulon - Bclaf1', 'Regulon - Bmyc', 'Regulon - Brf1', 'Regulon - Cdx2', 'Regulon - Creb1', 'Regulon - Ctcf', 'Regulon - Cux1', 'Regulon - Dbx1', 'Regulon - Dlx5', 'Regulon - Dlx6', 'Regulon - E2f1', 'Regulon - E2f2', 'Regulon - E2f3', 'Regulon - E2f4', 'Regulon - E2f5', 'Regulon - E2f6', 'Regulon - E2f7', 'Regulon - E2f8', 'Regulon - Elf1', 'Regulon - Elf2', 'Regulon - Elk3', 'Regulon - Emx2', 'Regulon - Esrra', 'Regulon - Ets1', 'Regulon - Ets2', 'Regulon - Etv6', 'Regulon - Evx1', 'Regulon - Ezh2', 'Regulon - Fli1', 'Regulon - Foxa1', 'Regulon - Foxa2', 'Regulon - Foxa3', 'Regulon - Foxc2', 'Regulon - Foxf1', 'Regulon - Foxo1', 'Regulon - Foxp2', 'Regulon - Gabpa', 'Regulon - Gata1', 'Regulon - Gata4', 'Regulon - Gata5', 'Regulon - Gata6', 'Regulon - Gbx2', 'Regulon - Gsx1', 'Regulon - Gtf2f1', 'Regulon - Gtf3

In [ ]:
nz_prop m= 0.1
scRNA= copute_local_mean(scRNA, representation='X_pca')
# scRNA = get_scVI_latent_representation(scRNA, cell_type_key='celltype', spatial_key='spatial')

# scRNA = clean_genes_for_LR_GSE232025(scRNA)
# scRNA = get_LR_pattern_representation(scRNA, nz_prop=nz_prop)

In [ ]:
def clean_genes(scRNA):
    scRNA.var["gene_symbol_original"] = scRNA.var_names
    def clean_name(name):
        # keep leftmost alias before '|'
        primary = name.split("|")[0]
        # remove [hs], [nr], etc.
        primary = re.sub(r"\[.*?\]", "", primary)
        return primary.strip()
    scRNA.var_names = [clean_name(x) for x in scRNA.var_names]
    scRNA.var_names_make_unique()
    return scRNA

In [ ]:
# scRNA= clean_genes(scRNA)
scRNA = scRNA = get_LR_pattern_representation(scRNA, nz_prop=nz_prop, resource_name='MouseConsensus')

Using `.X`!
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
Using resource `MouseConsensus`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/storage.py:48: FutureWarning: AnnData previously had undefined behavior around matrices of type <class 'scipy.sparse._coo.coo_matrix'>.In 0.12, passing in this type will throw an error. Please convert to a supported type.Continue using for this minor version at your own risk.
100%|██████████| 100/100 [00:01<00:00, 64.16it/s]
Using `.X`!
/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:381: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1 samples of mat are empty, they will be removed.
Using resource `MouseConsensus`.
/Users/rssantanu/anaconda3/envs/torch_flow_matching/

Total unique LR pairs across all time points: 9
Stage 0: 5913 cells, 8 LR pairs available
Stage 1: 18408 cells, 8 LR pairs available
Stage 2: 30124 cells, 7 LR pairs available
Stage 3: 51365 cells, 8 LR pairs available
Stage 4: 77369 cells, 8 LR pairs available
Stage 6: 113350 cells, 8 LR pairs available
Stage 7: 121767 cells, 7 LR pairs available
LR_pattern matrix shape: (418296, 9)
LR pair names stored in scRNA.uns['LR_pattern_pairs']


In [ ]:
scRNA.var

,highly_variable,highly_variable_rank,means,variances,variances_norm,highly_variable_nbatches
gene_short_name,,,,,,
Actc1,True,23.0,0.646842,0.733729,1.694188,7
Auts2,True,367.0,0.756789,0.608860,1.165527,7
Bnc2,True,845.0,0.153493,0.171893,1.008271,7
Calm1,True,1458.0,1.375792,0.694724,0.807404,7
Cd24a,True,845.0,1.084926,0.636479,0.952633,7
...,...,...,...,...,...,...
Olfr62,True,1121.0,0.000137,0.000137,1.050645,7
Gm47359,True,907.0,0.000122,0.000109,0.966791,7
Gm35326,True,761.0,0.000106,0.000100,1.018274,7
